###  Autor: Javier Garrucho Fernández

## Críticas de cine en IMDB

Los datos serán críticas de películas en la web IMDB (Internet Movie Database). Son críticas que ya vienen con la etiqueta "pos" o "neg", de acuerdo a la puntuación que acompaña a la crítica (positiva, 7 o más; negativa 4 o menos). El objetivo es ser capaz de declarar como positiva o negativa una crítica (por supuesto, sin saber la puntuación que la acompaña).

In [35]:
from sklearn.datasets import load_files

reviews_train = load_files("data/aclImdb/train/",encoding='utf-8')
text_train, y_train = reviews_train.data, reviews_train.target

reviews_test = load_files("data/aclImdb/test/",encoding='utf-8')
text_test, y_test = reviews_test.data, reviews_test.target

Quitamos algunas marcas en HTML, para "limpiar" los textos.

In [36]:
text_train = [doc.replace("<br />", " ") for doc in text_train]
text_test = [doc.replace("<br />", " ") for doc in text_test]

Para poder aplicar los clasificadores de Scikit Learn, hemos de vectorizar los textos. En scikitlearn se pueden elegir varias formas de vectorizar:

* **CountVectorizer**, modo binario (sólo se anota si un término ocurre o no)
* **CountVectorizer**, contando ocurrencias
* **TfIdfVectorizer**, vectorizando con TfIdf

El uso de estos vectorizadores se puede comprender a partir de lo visto en el tema de Procesamiento de Lenguaje Natural, y del manual de Scikit Learn. En particular **es importante el uso de los parámetros stop_words y min_df** para simplificar a vectorización. Una vez entendido y explorado su uso, elegir la mejor combinación de vectorizador y de clasificador, para este conjunto de datos. 

También mostrar la predicción que se realiza sobre algunas críticas concretas del conjunto de test. 

Empezaremos viendo el tamaño del conjunto de datos:

In [37]:
print(len(text_train),len(text_test))

25000 25000


Vemos que hay un total de 50000 reviews, y se utiliza la mitad para entrenamiento y la otra mitad para test.

Para las stop words en este caso utilizaré el conjunto que proporciona scikitlearn para el inglés, aunque podría ser interesante hacer uno más específico para este problema en concreto.

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

Creamos el vectorizador y vectorizamos los textos del conjunto de entrenamiento para obtener el vocabulario, lo guardamos en X_train_counts.

In [39]:
count_vect = CountVectorizer(min_df=1, stop_words="english")
X_train_counts = count_vect.fit_transform(text_train)

Para evitar que un texto por ser más grande y por tanto aparezca más veces una palabra que en uno más pequeño utilizamos la frecuencia del término, en vez de simplemente las veces que este aparece.

Para ello creamos un transformador a vector de pesos y se prepara para que pueda transformar otros vectores de ocurrencias en vectores de pesos, guardamos los vectores referentes al conjunto de entrenamiento en X_train_tfidf.

In [40]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

Ahora vectorizamos los textos del conjunto de test.

In [41]:
X_test_counts = count_vect.transform(text_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

Una vez que tanto los textos de test como de entrenamiento están representados correctamente en forma de vectores, ya podemos entrenar nuestro modelo y ver sus resultados, vamos a probar primero con los dos métodos vistos en clase para clasificación de texto, knn y Naive Bayes Multinomial.

Primero con **KNN**:

In [42]:
neigh_text = KNeighborsClassifier(n_neighbors=3).fit(X_train_tfidf, y_train)

In [43]:
pred_train = neigh_text.predict(X_train_tfidf)
pred_test = neigh_text.predict(X_test_tfidf)
print("Tasa de acierto sobre entrenamiento: {:.3f}".format(accuracy_score(y_train, pred_train)))
print("Tasa de acierto sobre test: {:.3f}".format(accuracy_score(y_test, pred_test)))
print("Matriz de confusión:\n{}".format(confusion_matrix(y_test, pred_test)))

Tasa de acierto sobre entrenamiento: 0.891
Tasa de acierto sobre test: 0.630
Matriz de confusión:
[[8161 4339]
 [4915 7585]]


KNN consigue relativamente buenos resultado en el conjunto de entrenamiento, sin embargo sobre el conjunto de test no clasifica demasiado bien.

Provemos ahora con **Naive Bayes Multinomial**.

In [44]:
from sklearn.naive_bayes import MultinomialNB

In [45]:
mnb_text = MultinomialNB().fit(X_train_tfidf, y_train)

In [46]:
pred_train = mnb_text.predict(X_train_tfidf)
pred_test = mnb_text.predict(X_test_tfidf)
print("Tasa de acierto sobre entrenamiento: {:.3f}".format(accuracy_score(y_train, pred_train)))
print("Tasa de acierto sobre test: {:.3f}".format(accuracy_score(y_test, pred_test)))
print("Matriz de confusión:\n{}".format(confusion_matrix(y_test, pred_test)))

Tasa de acierto sobre entrenamiento: 0.915
Tasa de acierto sobre test: 0.830
Matriz de confusión:
[[10961  1539]
 [ 2720  9780]]


Este método clasifica bastante mejor, aunque sigue teniendo un error grande.

Para determinar el mejor valor para min_df vamos a utilizar de nuevo **Grid-Search**, para hacerlo más compacto usaremos un **pipeline**, que reducirá los tres pasos (vectorización, transformación y entrenamiento) en una función.

**CIUDADO Esta función puede tardar varios minutos**

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [48]:
best_score=0
for min_df in [0.1, 0.2, 0.3, 1]:#Probamos estos valores para min_df (min_df=1 es por defecto)
        gs_txt_clf = text_clf = Pipeline([
         ('vect', CountVectorizer(min_df=min_df, stop_words="english")),
         ('tfidf', TfidfTransformer()),
         ('clf', MultinomialNB())
        ])
        # Realiza validación cruzada 
        scores = cross_val_score(gs_txt_clf, text_train, y_train, cv=5)
        # calcula el resultado medio de la validación cruzada
        score = np.mean(scores)
        # nos vamos quedando con la mejor combinación
        if score > best_score:
            best_score = score
            best_min_df = min_df
            
# Volvemos a entrenar el modelo con la mejor combinación encontrada, sobre entrenamieno+validación  
# y evaluamos el rendimiento sobre el conjunto de prueba 
gs_txt_clf = text_clf = Pipeline([
     ('vect', CountVectorizer(min_df=best_min_df, stop_words="english")),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB())
]).fit(text_train, y_test)

In [49]:
print("Mejor resultado sobre validación: {:.5f}".format(best_score))
print("Mejor min_df: ", best_min_df)
pred_test = gs_txt_clf.predict(text_test)
test_score = accuracy_score(y_test, pred_test)
print("Evaluación sobre el conjunto de test: {:.5f}".format(test_score))
print("Matriz de confusión:\n{}".format(confusion_matrix(y_test, pred_test)))

Mejor resultado sobre validación: 0.86548
Mejor min_df:  1
Evaluación sobre el conjunto de test: 0.82964
Matriz de confusión:
[[10961  1539]
 [ 2720  9780]]


El mejor valor obtenido para min_df ha sido 1. Con un score de 0.865 sobre el conjunto de validación y un porcentaje de aciertos del 83% sobre el conjunto de test.

Probemos con **regresión logística:**

De nuevo haciendo grid-dearch con validación cruzada para determinar ahora el valor para C.

**CIUDADO Esta función puede tardar varios minutos**

In [50]:
best_score=0
for C in [0.001, 0.01, 0.1, 1, 10, 100]:
    gs_txt_clf = text_clf = Pipeline([
     ('vect', CountVectorizer(min_df=1, stop_words="english")),
     ('tfidf', TfidfTransformer()),
     ('clf', LogisticRegression(max_iter=1000, C=C, penalty="l2"))
    ])
    # Realiza validación cruzada 
    scores = cross_val_score(gs_txt_clf, text_train, y_train, cv=5)
    # calcula el resultado medio de la validación cruzada
    score = np.mean(scores)
    # nos vamos quedando con la mejor combinación
    if score > best_score:
        best_score = score
        best_parameters = {'C': C}
        
gs_txt_clf = text_clf = Pipeline([
 ('vect', CountVectorizer(min_df=1, stop_words="english")),
 ('tfidf', TfidfTransformer()),
 ('clf', LogisticRegression(max_iter=1000,**best_parameters, penalty="l2"))
]).fit(text_train,y_train)


In [51]:
print("Mejor resultado sobre validación: {:.5f}".format(best_score))
print("Mejor combinación de valores: ", best_parameters)
pred_test = gs_txt_clf.predict(text_test)
test_score = accuracy_score(y_test, pred_test)
print("Evaluación sobre el conjunto de test: {:.5f}".format(test_score))
print("Matriz de confusión:\n{}".format(confusion_matrix(y_test, pred_test)))

Mejor resultado sobre validación: 0.89024
Mejor combinación de valores:  {'C': 10}
Evaluación sobre el conjunto de test: 0.87116
Matriz de confusión:
[[11012  1488]
 [ 1733 10767]]


Despues de todos los métodos probados regresión logística con C=10 es con el que mejor puntuación he obtenido, con un 89% de acierto sobre el conjunto de validación y un 87% sobre el conjunto de test.